In [1]:
import os, random, glob, pickle, collections, math
import numpy as np
import pandas as pd
import ujson as json
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
%matplotlib inline 

from keras.models import Sequential, Model, load_model, model_from_json
from keras.layers import GlobalAveragePooling2D, Flatten, Dropout, Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.preprocessing import image
from keras import backend as K
K.set_image_dim_ordering('tf')

Using TensorFlow backend.


In [2]:
TRAIN_DIR = '../data/train/'
CHECKPOINT_DIR = './checkpoints/checkpoint4/'
FISH_CLASSES = ['NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']
CONF_THRESH = 0.8
ROWS = 224
COLS = 224
BatchSize = 128
LearningRate = 1e-4

In [7]:
# GT_crop_bboxs_df = ['image_class','image_file','crop_index','crop_class','x_min',''y_min','x_max','ymax']

if os.path.exists('../data/GT_crop_bboxs_df_directory.pickle'):
    print ('Loading from file GT_crop_bboxs_df_directory.pickle.')
    with open('../data/GT_crop_bboxs_df_directory.pickle', 'rb') as f:
        GT_crop_bboxs_df = pd.read_pickle('GT_crop_bboxs_df_directory.pickle')
else:
    print ('Generating file GT_crop_bboxs_df_directory.pickle.'.format(ROWS, COLS))
    # like RCNN expand the crop by 16 pixels
    p=16        
    GT_crop_bboxs_df = pd.DataFrame(columns=['image_class','image_file','crop_index','crop_class','x_min','y_min','x_max','ymax'])  

    crop_classes=FISH_CLASSES[:]
    crop_classes.remove('NoF')

    for c in crop_classes:
        print(c)
        j = json.load(open('../data/BBannotations/{}.json'.format(c), 'r'))
        for l in j: 
            filename = l["filename"]
            head, tail = os.path.split(filename)
            basename, file_extension = os.path.splitext(tail) 
            image = Image.open(TRAIN_DIR+c+'/'+tail)
            width_image, height_image = image.size
            for i in range(len(l["annotations"])):
                a = l["annotations"][i]
                xmin = (a["x"])
                ymin = (a["y"])
                width = (a["width"])
                height = (a["height"])
                delta_width = p/(COLS-2*p)*width
                delta_height = p/(ROWS-2*p)*height
                xmin_expand = xmin-delta_width
                ymin_expand = ymin-delta_height
                xmax_expand = xmin+width+delta_width
                ymax_expand = ymin+height+delta_height
                assert max(xmin_expand,0)<min(xmax_expand,width_image)
                assert max(ymin_expand,0)<min(ymax_expand,height_image)
                GT_crop_bboxs_df.loc[len(GT_crop_bboxs_df)]=[c,tail,i,a["class"],max(xmin_expand,0),max(ymin_expand,0),min(xmax_expand,width_image),min(ymax_expand,height_image)]
                if a["class"] != c: print(GT_crop_bboxs_df.tail(1))


    #crop NoF by detections_full_AGNOSTICnms.pkl

    RFCN_MODEL = 'resnet101_rfcn_ohem_iter_30000'

    with open('../data/RFCN_detections/detections_full_AGNOSTICnms_'+RFCN_MODEL+'.pkl','rb') as f:
        detections_full_AGNOSTICnms = pickle.load(f, encoding='latin1') 
    train_detections_full_AGNOSTICnms = detections_full_AGNOSTICnms[1000:]
    with open("../RFCN/ImageSets/Main/train_test.txt","r") as f:
        train_files = f.readlines()
    assert len(train_detections_full_AGNOSTICnms) == len(train_files)


    num_NoF_perIm = 10

    for im in range(len(train_detections_full_AGNOSTICnms)):
        if im%1000 == 0: print(im)

        basename = train_files[im][:9]
        image_class = train_files[im][10:-1]
        image = Image.open(TRAIN_DIR+image_class+'/'+basename+'.jpg')
        width_image, height_image = image.size

        bboxes = []
        detects_im = train_detections_full_AGNOSTICnms[im]
        for i in range(len(detects_im)):
            if detects_im[i,4] >= 0.999 and detects_im[i,2]<width_image and detects_im[i,3]<height_image:
                bboxes.append(detects_im[i,:]) 
        bboxes = np.asarray(bboxes)
        bboxes = bboxes[np.random.choice(bboxes.shape[0], num_NoF_perIm, replace=False), :]

        for j in range(len(bboxes)):    
            bbox = bboxes[j]
            xmin = bbox[0]
            ymin = bbox[1]
            xmax = bbox[2]
            ymax = bbox[3]
            width = xmax-xmin
            height = ymax-ymin
            delta_width = p/(COLS-2*p)*width
            delta_height = p/(ROWS-2*p)*height
            xmin_expand = xmin-delta_width
            ymin_expand = ymin-delta_height
            xmax_expand = xmax+delta_width
            ymax_expand = ymax+delta_height
            assert max(xmin_expand,0)<min(xmax_expand,width_image)
            assert max(ymin_expand,0)<min(ymax_expand,height_image)
            GT_crop_bboxs_df.loc[len(GT_crop_bboxs_df)]=[image_class,basename+'.jpg',j,'NoF',max(xmin_expand,0),max(ymin_expand,0),min(xmax_expand,width_image),min(ymax_expand,height_image)]

    GT_crop_bboxs_df.to_pickle('../data/GT_crop_bboxs_df_directory.pickle')                

Loading from file GT_crop_bboxs_df_directory.pickle.


FileNotFoundError: [Errno 2] No such file or directory: 'GT_crop_bboxs_df_directory.pickle'

In [ ]:
def load_img(path, target_size=None):
    img = Image.open(path)
    img = img.convert('RGB')
    if target_size:
        img = img.resize((target_size[1], target_size[0]))
    return img

def preprocess_input(x):
    #resnet50 image preprocessing
    # 'RGB'->'BGR'
    x = x[:, :, ::-1]
    x[:, :, 0] -= 103.939
    x[:, :, 1] -= 116.779
    x[:, :, 2] -= 123.68
    return x

train_datagen = ImageDataGenerator(
    rotation_range=180,
    shear_range=0.2,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True)

nbr_perClass = int(BatchSize / len(FISH_CLASSES))   
def generator(datagen):
    while 1:
        batch_x = np.zeros((BatchSize, ROWS, COLS, 3), dtype=K.floatx())
        batch_y = np.zeros((BatchSize, len(FISH_CLASSES)), dtype=K.floatx())
        fn = lambda obj: obj.loc[np.random.choice(obj.index, size=nbr_perClass, replace=False),:]
        batch_df = GT_crop_bboxs_df.groupby('crop_class', as_index=True).apply(fn)
        i = 0
        for index,row in batch_df.iterrows():
            row = row.tolist()
            image_file = os.path.join(row[0], row[1])
            fish = row[3]
            bbox = row[4:8]
            img = load_img(TRAIN_DIR+image_file,target_size=(ROWS,COLS))
            x = np.asarray(img, dtype=K.floatx())
            x = datagen.random_transform(x)
            x = preprocess_input(x)
            batch_x[i] = x
            batch_y[i,FISH_CLASSES.index(fish)] = 1
            i += 1
        yield (batch_x, batch_y)
        
train_generator = generator(datagen = train_datagen)

In [ ]:
#Resnet50
#stg1 training

from keras.applications.resnet50 import ResNet50

base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
#x = Flatten()(x)
#x = Dense(256, init='glorot_normal', activation='relu')(x)
#x = LeakyReLU(alpha=0.33)(x)
#x = Dropout(0.5)(x)
#x = Dense(256, init='glorot_normal', activation='relu')(x)
#x = LeakyReLU(alpha=0.33)(x)
x = Dropout(0.5)(x)
predictions = Dense(len(FISH_CLASSES), init='glorot_normal', activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional VGG16 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
optimizer = Adam(lr=LearningRate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# train the model on the new data for a few epochs
model.fit_generator(train_generator, samples_per_epoch=BatchSize*2, nb_epoch=2, nb_worker=3, pickle_safe=True)